In [17]:
from keras.applications import vgg16
#Load the VGG model
vgg_conv = vgg16.VGG16(weights='imagenet', 
                 include_top=False, 
                 input_shape=(256, 256, 3))

In [18]:
# Freeze the layers except the last 3 layers
for layer in vgg_conv.layers[:-3]:
    layer.trainable = False
 

In [27]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Dense(1024, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(34, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 dense_9 (Dense)             (None, 8, 8, 1024)        525312    
                                                                 
 dropout_5 (Dropout)         (None, 8, 8, 1024)        0         
                                                                 
 dense_10 (Dense)            (None, 8, 8, 34)          34850     
                                                                 
Total params: 15,274,850
Trainable params: 5,279,778
Non-trainable params: 9,995,072
_________________________________________________________________


In [28]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [21]:
base_dir = '/content/imagedb_btsd.zip'

train_dir = os.path.join(base_dir, 'imagedb')
validation_dir = os.path.join(base_dir, 'imagedb_test')

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(256,256),
                                                    shuffle=True,
                                                    subset='training', seed=1)     
validation_generator =  train_datagen.flow_from_directory(train_dir,
                                                        batch_size=20,
                                                        class_mode='categorical',
                                                         target_size=(256,256),
                                                        subset='validation', seed=1) 

Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [30]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.adam_v2.Adam(learning_rate=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/30


ValueError: ignored

In [ ]:
test_datagen  = ImageDataGenerator(rescale=1./255)
test_dir = os.path.join(base_dir, 'test')
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                        batch_size=10,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(256,256)) 
loss, acc = model.evaluate(test_generator)

In [ ]:
from keras.preprocessing import image
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

path = '/content/animal_db_notebook_ready/test/penguin/158_0133.jpg'
img = image.load_img(path, target_size=(256, 256), grayscale=False, interpolation='bilinear')
plt.imshow(image.load_img(path), cmap='gray')
plt.show()

model = tf.keras.models.load_model('/content/small_last4.h5')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
  
classes_pred = model.predict(x)
classes = ['bear', 'camel', 'penguin']
print(f'\n\n\n{classes}')
print(f'\n\n\nSoftmax Output: {classes_pred}')
print(f'\n\n\n{path} is a {classes[classes_pred.argmax()]}\n\n\n')